We have the wave equation:
$$
\frac{\partial^{2} u}{\partial t^{2}} = c^{2}\nabla^{2} u
$$ 
defined over a domain $\Omega$. Let the boundary, $\partial \Omega$, but the union of the Neumann ($\Gamma_{N}$) and Dirichlet ($\Gamma_{D}$) boundaries.

Let boudnary conditions be:
$$
\begin{cases}
\nabla u \cdot n = 0 &\text{on }\Gamma_{N}\\
u = \cos(10\pi t)/10\pi &\text{on }\Gamma_{D}
\end{cases}
$$

The infinite dimension weak form is
$$
\int_{\Omega} w \frac{\partial^{2} u}{\partial t^{2}} \,dx = \int_{\Omega}c^{2} \nabla u \cdot \nabla w\,dx
$$
where $w$ is the weighting function in a suitable function space $V$.

We can turn this problem into a first order problem first. Let
$$
v = -\frac{\partial u}{\partial t}
$$
then,
$$
\begin{align}
&\frac{\partial u}{\partial t} = -v\\
&\frac{\partial v}{\partial t} + \nabla^{2} u = 0\\
&p = \sin(10\pi t) &&\text{on }\Gamma_{D}
\end{align}
$$
The weak form is
$$
\int_{\Omega} w \frac{\partial v}{\partial t} \,dx = \int_{\Omega} c^{2} \nabla u \cdot \nabla w\,dx
$$

In [93]:
from dolfin import *
from mshr import *

In [98]:
"""
Here we define the mesh
"""
plane_w = 2.0/3
plane_h = 8.0/3

wall_w = 0.001/3
wall_h = 0.05/3
slit_h = 0.5/3

plane = Rectangle(Point(plane_w*4, plane_h), Point(-plane_w, -plane_h))
wall1 = Rectangle(Point(wall_w, slit_h), Point(-wall_w, -slit_h))
wall2 = Rectangle(Point(wall_w, plane_h), Point(-wall_w, slit_h+wall_h))
wall3 = Rectangle(Point(wall_w, -plane_h), Point(-wall_w, -slit_h-wall_h))
wall4 = Rectangle(Point(-plane_w, plane_h), Point(-wall_w, slit_h*2))
wall5 = Rectangle(Point(-plane_w, -plane_h), Point(-wall_w, -slit_h*2))

domain = plane - wall1 - wall2 - wall3 - wall4 - wall5
mesh=generate_mesh(domain, 100)

In [99]:
"""
Define the weighting function and trial solution spaces
"""
order = 1 # order of basis functions

V = FunctionSpace(mesh, "CG", order)
a = TrialFunction(V)
phi = TestFunction(V)

intial_displacment = Constant(0.0)

intial_velocity = Constant(0.0)


u0 = interpolate(intial_displacment, V)
v0 = interpolate(intial_velocity, V)

In [100]:
def boundary(x, on_boundary):
    return on_boundary and near(x[0], -plane_w, DOLFIN_EPS)

# set the source to a constant for now
source = Expression("cos(10.0*pi*t)/(10.0*pi)", 
                    t = 0, 
                    degree = order)
bc = DirichletBC(V, source, boundary)

In [101]:
"""
Define the parameters for the newmark family
These correspond to the trapezoidal rule
"""
beta = 0.25
gamma = 0.5

t = 0
dt = 0.002
T = 10.0

def K_mat(function):
    return dot(grad(function), grad(phi))*dx

# assemble the variational problem
M = a*phi*dx # the mass matrix term, comes from the second derivative
K = K_mat(u0)
# F = interpolate(Constant(0.0), V)*dx # no forcing in the wave equation

a0 = Function(V)
solve(M == -K, a0, bc) # this gives us a0 to start our algorithm

# vtkFile_a = File("results/p{}_acc.pvd".format(order))
# vtkFile_a << a0

vtkFile_u = File("results/p{}_displacement.pvd".format(order))
vtkFile_u << u0

# general lefthand using Newmark family, for linear hyperbolic equations
A = assemble(M + dt*dt*beta*K_mat(a))
L = -(K_mat(u0) + dt*K_mat(v0) + dt*dt*(1-2*beta)*K_mat(a0))

          Calling FFC just-in-time (JIT) compiler, this may take some time.
          Calling FFC just-in-time (JIT) compiler, this may take some time.


In [102]:
a1 = Function(V) # acceleration at t_(n+1)

step = 0
while t <= T:
    # update time and step
    t += dt
    step += 1
    source.t = t
    
    b = assemble(L)
    bc.apply(A,b)

    solve(A, a1.vector(), b) # solves for a_1
    
    u1 = u0 + dt*v0 + dt*dt*(1-2*beta)*a0 + dt*dt*beta*a1
    v1 = v0 + dt*(1-gamma)*a0 + dt*gamma*a1
    
    u0.assign(u1)
    v0.assign(v1)
    a0.assign(a1)

    if step % 10 == 0:
        vtkFile_u << (u0, t)

          Calling FFC just-in-time (JIT) compiler, this may take some time.
